In [3]:
%reload_ext autoreload

from config.Neo_connect import Neo4jConnection
import pandas as pd
import config.db_settings as db_config

# stablish connection with neo4j
conn = Neo4jConnection(uri="bolt://localhost:7687", user=db_config.USER, pwd=db_config.PASSWORD)
#result = conn.query('create database {}'.format(DATABASE))



In [10]:
#create nodes

#-----------------------------------
node_queries = [
'''
LOAD CSV WITH HEADERS FROM 'file:/C:/Users/ProBook/anaconda3/upc/sdm/SDM_Project/data/stock_data.csv' as row FIELDTERMINATOR ',' CREATE (:stock {symbol:row.symbol,lastPrice :row.lastPrice,
dayHigh: row.dayHigh,dayLow: row.dayLow,closePrice:row.closePrice,open:row.open,timestamp_ID:row.timestamp_ID, stock_market_ID:row.stock_market_ID,
company_ID:row.company_ID }) return row.symbol;
''',
'''
LOAD CSV WITH HEADERS FROM 'file:/C:/Users/ProBook/anaconda3/upc/sdm/SDM_Project/data/stock_data.csv' as row FIELDTERMINATOR ',' CREATE (:company {company_ID:row.company_ID,symbol:row.symbol,companyName:row.companyName }) return row.company_ID;
''',

'''
LOAD CSV WITH HEADERS FROM 'file:/C:/Users/ProBook/anaconda3/upc/sdm/SDM_Project/data/stock_market.csv' as row FIELDTERMINATOR ',' CREATE (:stock_market {ID:row.ID, country:row.country, stock_name:row.stock_name }) return row.country;
''',
'''
LOAD CSV WITH HEADERS FROM 'file:/C:/Users/ProBook/anaconda3/upc/sdm/SDM_Project/data/time.csv' as row FIELDTERMINATOR ',' CREATE (:time {ID:row.ID, year:row.year, month:row.month, day:row.day, hour:row.hour, timestamp:row.timestamp}) return row.ID;
''',
'''
LOAD CSV WITH HEADERS FROM 'file:/C:/Users/ProBook/anaconda3/upc/sdm/SDM_Project/data/tweets_users.csv' as row FIELDTERMINATOR ',' CREATE (:tweets {tweet_ID:row.tweet_ID,symbol:row.company_name,text:row.text,support_Count:row.support_Count,rank:row.rank,sentiment:row.sentiment,polarity:row.polarity,company_ID:row.company_ID,timestamp_ID:row.timestamp_ID}) return row.tweet_ID;
''',
'''
LOAD CSV WITH HEADERS FROM 'file:/C:/Users/ProBook/anaconda3/upc/sdm/SDM_Project/data/tweets_users.csv' as row FIELDTERMINATOR ',' CREATE (:tweet_users {tweet_ID:row.tweet_ID, user: row.user,mentioned_user:row.mentioned_user}) return row.user;
''']


for i, nd_query in enumerate(node_queries):
    result = conn.query(nd_query, db=db_config.DATABASE)
    print("query {} executed \n {} \n".format(i+1,nd_query))


query 1 executed 
 
LOAD CSV WITH HEADERS FROM 'file:/C:/Users/ProBook/anaconda3/upc/sdm/SDM_Project/data/tweets_users.csv' as row FIELDTERMINATOR ',' CREATE (:tweets {tweet_ID:row.tweet_ID,symbol:row.company_name,text:row.text,support_Count:row.support_Count,rank:row.rank,sentiment:row.sentiment,polarity:row.polarity,company_ID:row.company_ID,timestamp_ID:row.timestamp_ID}) return row.tweet_ID;
 

query 2 executed 
 
LOAD CSV WITH HEADERS FROM 'file:/C:/Users/ProBook/anaconda3/upc/sdm/SDM_Project/data/tweets_users.csv' as row FIELDTERMINATOR ',' CREATE (:tweet_users {tweet_ID:row.tweet_ID, user: row.user,mentioned_user:row.mentioned_user}) return row.user;
 



In [11]:


#make_relations
#-----------------------------------------------

relation_queries = [
'''
MATCH (st:stock),(company:company)
where st.company_ID = company.company_ID 
CREATE (st)-[r:belongs_to]->(company) 
return r
''',
'''
MATCH (t:tweets),(company:company)
where t.company_ID = company.company_ID 
CREATE (st)-[r:is_related_to]->(company) 
return r

''',
'''
MATCH (st:stock),(t:time)
where st.timestamp_ID = t.timestamp_ID 
CREATE (st)-[r:at]->(t) 
return r

''',
'''
MATCH (t:tweets),(time: time)
where t.timestamp_ID = time.ID
CREATE (t)-[r:at_this]->(time) 
return r
 
''',
'''
MATCH (s:stock),(sm: stock_market)
where s.stock_market_ID = sm.ID
CREATE (s)-[r:is_in]->(sm) 
return r

''',    
'''
MATCH (t:tweets),(tu: tweet_users)
where t.tweet_ID = tu.tweet_ID
CREATE (t)-[r:contains]->(tu) 
return r

''']
    
# '''
# MATCH (art:article {id: row.article_ID}),(publ:conference {id: row.publisher_ID})
# CREATE (art)-[r:is_published_in{year:row.year}]->(publ) return r''',

# '''LOAD CSV WITH HEADERS FROM 'file:/SDM_lab1/kaggle_data_processed/article_published_by.csv' as row FIELDTERMINATOR ','
# MATCH (art:article {id: row.article_ID}),(publ:journal {id: row.publisher_ID})
# CREATE (art)-[r:is_published_in{year:row.year, volume:"1"}]->(publ) return r'''
# '''
# LOAD CSV WITH HEADERS FROM 'file:/SDM_lab1/kaggle_data_processed/paper_cited_by.csv' as row FIELDTERMINATOR ','
# MATCH (art:article {id: row.article_ID}),(cite:article {id: row.cite_paper_ID})
# CREATE (art)-[r:has_cited{year:row.year, volume:"1"}]->(cite) return r
# ''',
# '''
# LOAD CSV WITH HEADERS FROM 'file:/SDM_lab1/kaggle_data_processed/article_has_keyword.csv' as row FIELDTERMINATOR ','
# MATCH (art:article {id: row.article_ID}),(key:Keyword {id: row.keyword_ID})
# CREATE (art)-[r:has]->(key) return r
# ''']


for j,re_query in enumerate(relation_queries):
    result = conn.query(re_query, db=db_config.DATABASE)
    print("query {} executed \n {} \n".format(j+1,re_query))




#=======================================================================================
# # creating index on publisher node
# ind_q = '''CREATE INDEX conf_journ_idx FOR (pub:publisher) ON (pub.type)'''
# result = conn.query(ind_q, db=db_config.DATABASE)
# print("index created on publisher node on attribute type")












# q = ''' MATCH (n:article) RETURN n LIMIT 25'''
# result = conn.query(q, db=DATABASE)
# # print(result[1].data())
# dtf_data = pd.DataFrame([_.data()['n'] for _ in result])
# print(dtf_data)
# loading data from CSV
#---------------------------------------------------------------------


query 1 executed 
 
MATCH (st:stock),(company:company)
where st.company_ID = company.company_ID 
CREATE (st)-[r:belongs_to]->(company) 
return r
 

query 2 executed 
 
MATCH (t:tweets),(company:company)
where t.company_ID = company.company_ID 
CREATE (st)-[r:is_related_to]->(company) 
return r

 

query 3 executed 
 
MATCH (st:stock),(t:time)
where st.timestamp_ID = t.timestamp_ID 
CREATE (st)-[r:at]->(t) 
return r

 

query 4 executed 
 
MATCH (t:tweets),(time: time)
where t.timestamp_ID = time.ID
CREATE (t)-[r:at_this]->(time) 
return r
 
 

query 5 executed 
 
MATCH (s:stock),(sm: stock_market)
where s.stock_market_ID = sm.ID
CREATE (s)-[r:is_in]->(sm) 
return r

 

query 6 executed 
 
MATCH (t:tweets),(tu: tweet_users)
where t.tweet_ID = tu.tweet_ID
CREATE (t)-[r:contains]->(tu) 
return r

 

